In [4]:
import pyreadstat

df_path="../dataset/raw/01/label SAK202408.sav"
# Read the .sav file
df, meta = pyreadstat.read_sav(df_path, encoding='latin1')

df.head()
# Save as .csv
#df.to_csv("../dataset/raw/01/label_SAK2002408_01.csv", index=False)


,tahun,r13a,r13b,r13c,r15a,r15b,r30a,r30b,r30c,r33a,...,r47a,r47b,r47c,r48a,r48b,r55a,r55b,r55c,r56a,r56b
0,20248.0,Menganyam tali serabut,Tali serabut,Industri tali serabut,13941.0,7317.0,,,,NaN,...,,,,NaN,NaN,,,,NaN,NaN
1,20248.0,Membuat Lemari kaca Stainless,Lemari kaca,Membuat Lemari Stainless (Kaca) Bersama Saudar...,25992.0,7522.0,,,,NaN,...,,,,NaN,NaN,,,,NaN,NaN
2,20248.0,Memasang payet Pada baju,Payet Baju,"Memasang payet baju Satuan, Dibayar Setelah SE...",14120.0,7533.0,,,,NaN,...,,,,NaN,NaN,,,,NaN,NaN
3,20248.0,Melakukan pekerjaan proyek bangunan rumah,Jasa konstruksi bangunan rumah,Kontruksi bangunan rumah,41011.0,9313.0,,,,NaN,...,membuat teras rumah,bangunan rumah,kontruksi rumah,41011.0,9313.0,,,,NaN,NaN
4,20248.0,Membersihkan gulma di padi sawah milik sendiri,Padi,Pertanian tanaman pangan milik sendiri,1122.0,6111.0,,,,NaN,...,,,,NaN,NaN,,,,NaN,NaN


In [9]:
import pandas as pd
import os
## load data
main_path="../dataset/raw/"

os.mkdir("../dataset/raw/csv")



In [11]:
import pyreadstat

path="../dataset/raw"
print(path)
for file in os.listdir(path):
    if file.endswith(".sav"):
        print(file)
        file_path=path+"/"+file
        df, meta = pyreadstat.read_sav(file_path,encoding='latin1')
        if not os.path.exists(path+"/csv"):
            os.makedirs(path+"/csv")
        df.to_csv(path+"/csv/"+file[:-4]+".csv")
        print("Done")
    else:
        continue
            



../dataset/raw
label SAK202108.sav
Done
label SAK202208.sav
Done
label SAK202308.sav
Done
label SAK202408.sav
Done


## restructure the data

In [23]:
def map_concat_columns(row,list_text_cols):
    aktivitas = row[list_text_cols[0]] if pd.notnull(row[list_text_cols[0]]) else ''
    produk = row[list_text_cols[1]] if pd.notnull(row[list_text_cols[1]]) else ''
    usaha = row[list_text_cols[2]] if pd.notnull(row[list_text_cols[2]]) else ''
    return f"Aktivitas: {aktivitas}. Produk: {produk}. Usaha: {usaha}."



def cleaning_dataset(df,cols_old,kbli_code_col,kbji_code_col,text_description_col):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    
    df = df.dropna(subset=[kbli_code_col])  # kbli_code_col in string
    df = df.dropna(subset=[kbji_code_col])  # kbji_code_col in string
    df_new = df[cols_old].copy()  # cols_old in list
    q_df = df_new[text_description_col]
    
    text_description = q_df.apply(lambda row: map_concat_columns(row, list_text_cols=text_description_col), axis=1)
    kbli_codes=df_new[kbli_code_col]
    kbji_codes=df_new[kbji_code_col]

    df_new['text_description'] = text_description
    #lowercase
    df_new['text_description'] = df_new['text_description'].str.lower()

    df_new['kbli_code'] = kbli_codes
    #convert kbli code to int
    df_new['kbli_code'] = df_new['kbli_code'].astype(int)
    df_new['kbli_code'] = df_new['kbli_code'].astype(str)
    df_new['kbli_code'] = df_new['kbli_code'].str.zfill(5)

    df_new['kbji_code'] = kbji_codes
    #convert kbji code to int
    df_new['kbji_code'] = kbji_codes
    df_new['kbji_code'] = df_new['kbji_code'].astype(int)
    df_new['kbji_code'] = df_new['kbji_code'].astype(str)
    df_new['kbji_code'] = df_new['kbji_code'].str.zfill(4)


    clean_df = df_new[['text_description','kbli_code','kbji_code']]
    return clean_df

### 2021

In [21]:
import pandas as pd

path = "../dataset/raw/csv"

df_path = path + "/label SAK202108.csv"

# Read the columns as floats first
df = pd.read_csv(df_path, dtype={"R13A": float, "R13B": float})


# Convert to integers if possible
#df["R13A"] = df["R13A"].astype(int, errors='ignore')
#df["R13B"] = df["R13B"].astype(int, errors='ignore')



print(df.columns)
df.head()

Index(['Unnamed: 0', 'tahun', 'R11A', 'R11B', 'R11C', 'R13A', 'R13B', 'R36A',
       'R36B', 'R36C', 'R37A', 'R37B'],
      dtype='object')


,Unnamed: 0,tahun,R11A,R11B,R11C,R13A,R13B,R36A,R36B,R36C,R37A,R37B
0,0,20218.0,KEPALA DESA,JASA PEMERINTAHAN,PEMERINTAHAN,84119.0,1113.0,NaN,NaN,NaN,NaN,NaN
1,1,20218.0,MENCANGKUL DAN MENANAM PADI,PADI,PERTANIAN PADI DI SAWAH MILIK SENDIRI,1122.0,6111.0,NaN,NaN,NaN,NaN,NaN
2,2,20218.0,MELAYANI ADMINISTRASI DI KANTOR KECAMATAN,KASUBAK/JASA PEMERINTAHAN,PNS/PEMERINTAHAN,84119.0,1213.0,NaN,NaN,NaN,NaN,NaN
3,3,20218.0,ADMINISTRASI SMP 1 NEGERI TEUPAH SELATAN,TATA USAHA/JASA PEMERINTAHAN,KONTRAK/PEMERINTAHAN,85491.0,4131.0,NaN,NaN,NaN,NaN,NaN
4,4,20218.0,JUALAN BAHAN MAKANAN,SEMBAKO,PERDAGANGAN,47112.0,5211.0,NaN,NaN,NaN,NaN,NaN


In [24]:
cols_old=['R11A', 'R11B', 'R11C', 'R13A', 'R13B']
kbli_codes='R13A'
kbji_codes='R13B'
text_description_col=['R11A', 'R11B', 'R11C']

df_2021=cleaning_dataset(df=df,cols_old=cols_old,kbli_code_col=kbli_codes,kbji_code_col=kbji_codes,text_description_col=text_description_col)


## 2022

In [ ]:
import pandas as pd

path = "../dataset/raw/csv"

df_path = path + "/label SAK202208.csv"

df = pd.read_csv(df_path)
print(df.columns)
df.head()

Index(['Unnamed: 0', 'tahun', 'R12A', 'R12B', 'R12C', 'R14A', 'R14B', 'R40A',
       'R40B', 'R40C', 'R41A', 'R41B'],
      dtype='object')


,Unnamed: 0,tahun,R12A,R12B,R12C,R14A,R14B,R40A,R40B,R40C,R41A,R41B
0,0,20228.0,PNS SEKDES DI DESA ANAAO,JASA PEMERINTAHAN,PEMERINTAHAN DESA,84119.0,4110.0,NaN,NaN,NaN,NaN,NaN
1,1,20228.0,MEMANCING DILAUT MENGGUNAKAN PERAHU SENDIRI,IKAN,PERIKANAN PANCING,3111.0,6340.0,NaN,NaN,NaN,NaN,NaN
2,2,20228.0,MENCARI GURITA DILAUT LEPAS DENGAN CARA BERENA...,"GURITA, IKAN, DLL",PERIKANAN TANGKAP,3113.0,6340.0,NaN,NaN,NaN,NaN,NaN
3,3,20228.0,MENANAM PADI DI SAWAH,PADI,PERTANIAN PADI DI SAWAH,1122.0,6111.0,NaN,NaN,NaN,NaN,NaN
4,4,20228.0,MEMBUAT KUE DAN DIJUAL DIWARUNG,KUE,PENYEDIAAN MAKANAN,10710.0,5249.0,NaN,NaN,NaN,NaN,NaN


In [26]:
cols_old=['R12A', 'R12B', 'R12C', 'R14A', 'R14B']
kbli_codes=cols_old[3]
kbji_codes=cols_old[4]
text_description_col=cols_old[:3]

df_2022=cleaning_dataset(df=df,cols_old=cols_old,kbli_code_col=kbli_codes,kbji_code_col=kbji_codes,text_description_col=text_description_col)
df_2022.head()

,text_description,kbli_code,kbji_code
0,aktivitas: pns sekdes di desa anaao. produk: j...,84119,4110
1,aktivitas: memancing dilaut menggunakan perahu...,03111,6340
2,aktivitas: mencari gurita dilaut lepas dengan ...,03113,6340
3,aktivitas: menanam padi di sawah. produk: padi...,01122,6111
4,aktivitas: membuat kue dan dijual diwarung. pr...,10710,5249


## 2023

In [28]:
import pandas as pd

path = "../dataset/raw/csv"

df_path = path + "/label SAK202308.csv"

df = pd.read_csv(df_path)
print(df.columns)
df.head()

Index(['Unnamed: 0', 'tahun', 'R12A', 'R12B', 'R12C', 'R14A', 'R14B', 'R40A',
       'R40B', 'R40C', 'R41A', 'R41B', 'R48A', 'R48B', 'R48C', 'R49', 'R50'],
      dtype='object')


C:\Users\Najwan\AppData\Local\Temp\ipykernel_22036\457549491.py:7: DtypeWarning: Columns (12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path)


,Unnamed: 0,tahun,R12A,R12B,R12C,R14A,R14B,R40A,R40B,R40C,R41A,R41B,R48A,R48B,R48C,R49,R50
0,0,20238.0,MEMANGKAS RAMBUT,JASA PANGKAS,TUKANG PANGKAS,96111.0,5141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20238.0,MENANGKAP GURITA DENGAN CARA BERENANG BEBAS,GURITA DAN LOBSTER,NELAYAN TANGKAP,3111.0,6222.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,20238.0,MENANAM PADI,PADI,PETANI SAWAH,1122.0,6111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,20238.0,MEMANEN BUAH KELAPA,BUAH KELAPA,PERKEBUNAN,1261.0,6112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,20238.0,MENANAM DAN MEMANEN SAYURSAYURAN,SAYURSAYURAN,PETANI SAYUR,1131.0,6111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
cols_old=['R12A', 'R12B', 'R12C', 'R14A', 'R14B']
kbli_codes=cols_old[3]
kbji_codes=cols_old[4]
text_description_col=cols_old[:3]

df_2023=cleaning_dataset(df=df,cols_old=cols_old,kbli_code_col=kbli_codes,kbji_code_col=kbji_codes,text_description_col=text_description_col)
df_2023.head()

,text_description,kbli_code,kbji_code
0,aktivitas: memangkas rambut. produk: jasa pang...,96111,5141
1,aktivitas: menangkap gurita dengan cara berena...,03111,6222
2,aktivitas: menanam padi. produk: padi. usaha: ...,01122,6111
3,aktivitas: memanen buah kelapa. produk: buah k...,01261,6112
4,aktivitas: menanam dan memanen sayursayuran. p...,01131,6111


## 2024

In [30]:
import pandas as pd

path = "../dataset/raw/csv"

df_path = path + "/label SAK202408.csv"

df = pd.read_csv(df_path)
print(df.columns)
df.head()

Index(['Unnamed: 0', 'tahun', 'r13a', 'r13b', 'r13c', 'r15a', 'r15b', 'r30a',
       'r30b', 'r30c', 'r33a', 'r33b', 'r47a', 'r47b', 'r47c', 'r48a', 'r48b',
       'r55a', 'r55b', 'r55c', 'r56a', 'r56b'],
      dtype='object')


,Unnamed: 0,tahun,r13a,r13b,r13c,r15a,r15b,r30a,r30b,r30c,...,r47a,r47b,r47c,r48a,r48b,r55a,r55b,r55c,r56a,r56b
0,0,20248.0,Menganyam tali serabut,Tali serabut,Industri tali serabut,13941.0,7317.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20248.0,Membuat Lemari kaca Stainless,Lemari kaca,Membuat Lemari Stainless (Kaca) Bersama Saudar...,25992.0,7522.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,20248.0,Memasang payet Pada baju,Payet Baju,"Memasang payet baju Satuan, Dibayar Setelah SE...",14120.0,7533.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,20248.0,Melakukan pekerjaan proyek bangunan rumah,Jasa konstruksi bangunan rumah,Kontruksi bangunan rumah,41011.0,9313.0,NaN,NaN,NaN,...,membuat teras rumah,bangunan rumah,kontruksi rumah,41011.0,9313.0,NaN,NaN,NaN,NaN,NaN
4,4,20248.0,Membersihkan gulma di padi sawah milik sendiri,Padi,Pertanian tanaman pangan milik sendiri,1122.0,6111.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
cols_old=['r13a', 'r13b', 'r13c', 'r15a', 'r15b']
kbli_codes=cols_old[3]
kbji_codes=cols_old[4]
text_description_col=cols_old[:3]

df_2024=cleaning_dataset(df=df,cols_old=cols_old,kbli_code_col=kbli_codes,kbji_code_col=kbji_codes,text_description_col=text_description_col)
df_2024.head()

,text_description,kbli_code,kbji_code
0,aktivitas: menganyam tali serabut. produk: tal...,13941,7317
1,aktivitas: membuat lemari kaca stainless. prod...,25992,7522
2,aktivitas: memasang payet pada baju. produk: p...,14120,7533
3,aktivitas: melakukan pekerjaan proyek bangunan...,41011,9313
4,aktivitas: membersihkan gulma di padi sawah mi...,01122,6111


In [34]:
#combine df_2021 df_2022 df_2023 df_2024
import csv
df_output=pd.concat([df_2021,df_2022,df_2023,df_2024],axis=0)
df_output.head()
df_output.to_csv("../dataset/clean/clean_sak_occupation_full.csv",index=False, sep=',', quotechar='"', quoting=csv.QUOTE_ALL, encoding='utf-8')

### loada Data

In [ ]:
df_loaded = pd.read_csv('../dataset/clean/clean_sak_occupation_full.csv', 
                        sep=',', 
                        quotechar='"', 
                        encoding='utf-8')
df_loaded.head()